In [ ]:
import pandas as pd

In [ ]:
publication = pd.read_excel('/content/Publications17-21.xlsx', header = 0, na_values='-' )


Se separan los autores y las materias y se ponen en lineas por separado

In [ ]:
publication['Scopus Author Ids'] = publication['Scopus Author Ids'].str.split('|')
publication['Quacquarelli Symonds (QS) Subject field name'] = publication['Quacquarelli Symonds (QS) Subject field name'].str.split('|')

In [ ]:
pub = publication.explode('Scopus Author Ids', ignore_index=True)

In [ ]:
pub = pub.explode('Quacquarelli Symonds (QS) Subject field name', ignore_index=True)

In [ ]:
pub.shape

(2029066, 50)

Se va a trabajar solo con los de Physics & Astronomy

In [ ]:
grouped = pub.groupby(['Quacquarelli Symonds (QS) Subject field name'])
Physics_Astronomy = pd.DataFrame(grouped.get_group('Physics & Astronomy'))
Physics_Astronomy2 = pd.DataFrame(grouped.get_group(' Physics & Astronomy'))

In [ ]:
Physics_Astronomy.append(Physics_Astronomy2)
Physics_Astronomy.reset_index(drop=True, inplace=True)

Quitamos las columnas que no nos aportan información valiosa a nuestro análisis

In [ ]:
Physics_Astronomy.drop(['PubMed ID','Open Access','Volume', 'Issue', 'Pages', 'Sustainable Development Goals (2022)', 'Scopus Author ID Single Author', 'Scopus Author ID Corresponding Author'], inplace = True, axis = 1)

In [ ]:
Physics_Astronomy.drop(['Number of Authors','Full date','Source ID','Scopus Source title','Article number','Number of Institutions','Scopus Author ID Last Author','All Science Journal Classification (ASJC) code','Quacquarelli Symonds (QS) Subject area code','Quacquarelli Symonds (QS) Subject code','Time Higher Education (THE) code','Topic Cluster number','Topic number'], inplace = True, axis = 1)

In [ ]:
Physics_Astronomy.shape

(752597, 29)

In [ ]:
Physics_Astronomy.drop(['Publication type','All Science Journal Classification (ASJC) field name','Quacquarelli Symonds (QS) Subject area field name','Quacquarelli Symonds (QS) Subject field name','Time Higher Education (THE) field name','Topic Cluster name','Topic name','Patent citations'], inplace = True, axis = 1)

Hacemos un mapa de correlación de datos para cononcer mejor los datos

Nos quedaremos solo con los Journals, aún así queda una gran cantidad de datos (742014)

In [ ]:
Physics_Astronomy_Journal = Physics_Astronomy.loc[Physics_Astronomy.loc[:, 'Source type'] == 'Journal']

In [ ]:
Physics_Astronomy_Journal.shape

(742014, 21)

In [ ]:
Physics_Astronomy_Journal.isna().sum()

Scopus Author Ids                                                       192
Year                                                                      0
ISSN                                                                      0
Source type                                                               0
SNIP (publication year)                                                 256
SNIP percentile (publication year) *                                    256
CiteScore (publication year)                                            389
CiteScore percentile (publication year) *                               389
SJR (publication year)                                                  714
SJR percentile (publication year) *                                     714
Field-Weighted View Impact                                                0
Views                                                                     0
Citations                                                                 0
Field-Weight

Borramos las filas que no tengan información en el Scopus Author Ids, ya que estas no se pueden reemplazar con ningun otro valor

In [ ]:
Physics_Astronomy_Journal = Physics_Astronomy_Journal[Physics_Astronomy_Journal['Scopus Author Ids'].notna()]

Los demás serán reemplazados con la media de cada una de las columnas

In [ ]:
Physics_Astronomy_Journal['SNIP (publication year)'] = Physics_Astronomy_Journal['SNIP (publication year)'].fillna(Physics_Astronomy_Journal['SNIP (publication year)'].mean())
Physics_Astronomy_Journal['SNIP percentile (publication year) *'] = Physics_Astronomy_Journal['SNIP percentile (publication year) *'].fillna(Physics_Astronomy_Journal['SNIP percentile (publication year) *'].mean())
Physics_Astronomy_Journal['CiteScore (publication year)'] = Physics_Astronomy_Journal['CiteScore (publication year)'].fillna(Physics_Astronomy_Journal['CiteScore (publication year)'].mean())
Physics_Astronomy_Journal['CiteScore percentile (publication year) *'] = Physics_Astronomy_Journal['CiteScore percentile (publication year) *'].fillna(Physics_Astronomy_Journal['CiteScore percentile (publication year) *'].mean())
Physics_Astronomy_Journal['SJR (publication year)'] = Physics_Astronomy_Journal['SJR (publication year)'].fillna(Physics_Astronomy_Journal['SJR (publication year)'].mean())
Physics_Astronomy_Journal['SJR percentile (publication year) *'] = Physics_Astronomy_Journal['SJR percentile (publication year) *'].fillna(Physics_Astronomy_Journal['SJR percentile (publication year) *'].mean())
Physics_Astronomy_Journal['Topic Cluster Prominence Percentile'] = Physics_Astronomy_Journal['Topic Cluster Prominence Percentile'].fillna(Physics_Astronomy_Journal['Topic Cluster Prominence Percentile'].mean())
Physics_Astronomy_Journal['Topic Prominence Percentile'] = Physics_Astronomy_Journal['Topic Prominence Percentile'].fillna(Physics_Astronomy_Journal['Topic Prominence Percentile'].mean())

In [ ]:
Physics_Astronomy_Journal.isna().sum()

Scopus Author Ids                                                     0
Year                                                                  0
ISSN                                                                  0
Source type                                                           0
SNIP (publication year)                                               0
SNIP percentile (publication year) *                                  0
CiteScore (publication year)                                          0
CiteScore percentile (publication year) *                             0
SJR (publication year)                                                0
SJR percentile (publication year) *                                   0
Field-Weighted View Impact                                            0
Views                                                                 0
Citations                                                             0
Field-Weighted Citation Impact                                  

In [ ]:
corr = Physics_Astronomy.corr()
corr.style.background_gradient (cmap = 'coolwarm')

#Modelo

definición de SNIP: https://journalinsights.elsevier.com/journals/0969-806X/snip

Consideramos que el SNIP es una de las métricas con un mayor valor en cuanto publicaciones, además es una métrica que modifica su valor dependiendo a la materia de la publicación. Además, vimos en el mapa de correlación que muchas métricas tienen una relación buena con el SNIP. Por estos factores la elegimos como la y a predecir.

In [ ]:
X = Physics_Astronomy_Journal[['SNIP percentile (publication year) *','CiteScore (publication year)','CiteScore percentile (publication year) *','SJR (publication year)','SJR percentile (publication year) *','Field-Weighted View Impact','Views','Citations','Field-Weighted Citation Impact','Field-Citation Average','Outputs in Top Citation Percentiles, per percentile','Field-Weighted Outputs in Top Citation Percentiles, per percentile','Topic Cluster Prominence Percentile','Topic Prominence Percentile']].values
y = Physics_Astronomy_Journal['SNIP (publication year)'].values

Como tenemos muchos datos, se elige un modelo de 80% para el train y 20% para el test

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=.30,random_state=0)

Primero se hace el modelo de regresión lineal

In [ ]:
from sklearn.linear_model import LinearRegression
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

El puntaje que sale es bueno, sin embargo probaremos con otro método para encontrar el mejor

In [ ]:
predictions = modelo.predict(X_test)

print('Puntaje entrenamiento: {}\n'.format(modelo.score(X_train,y_train)))
print('Puntaje Test: {}\n'.format(modelo.score(X_test,y_test)))

Puntaje entrenamiento: 0.9068013588553334

Puntaje Test: 0.9043344193161464



Al final, el mejor modelo resultó ser el kNN, que tiene un fantástico puntaje tanto de entrenamiento como de test

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
modelKNN = KNeighborsRegressor(5,metric='euclidean')
modelKNN.fit(X_train,y_train)
predictionsKNN = modelKNN.predict(X_test)
y_pred = modelKNN.predict(X_test)

print('Puntaje entrenamiento: {}\n'.format(modelKNN.score(X_train,y_train)))
print('Puntaje Test: {}\n'.format(modelKNN.score(X_test,y_test)))

Puntaje entrenamiento: 0.9993100751803708

Puntaje Test: 0.997683365846829



Para hacer una validación, se hace una tabla con los valores del SNIP, lo que se predijo y la diferencia, además se imprime abajo la descripción de las diferencias donde se puede notar que es una muy buena predicción.

In [ ]:
validacion = pd.DataFrame({'Actual': y_test, 'Predicción': predictions, 'Diferencia': y_test-predictions})
muestra_validacion = validacion.head()
muestra_validacion

,Actual,Predicción,Diferencia
0,1.47,1.455938,0.014062
1,0.78,0.803988,-0.023988
2,1.28,1.425704,-0.145704
3,1.23,1.236121,-0.006121
4,1.24,1.266619,-0.026619


In [ ]:
validacion["Diferencia"].describe()

count    148365.000000
mean         -0.000432
std           0.163271
min          -1.332808
25%          -0.063832
50%          -0.004329
75%           0.050968
max           5.167882
Name: Diferencia, dtype: float64